In [1]:
# webdriver_manager로 자동으로 열기
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
# 자동화하는 라이브러리
from selenium import webdriver

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())  # ChromeDriver 경로 자동 설정
driver = webdriver.Chrome(service=service)

URL = "https://www.starbucks.co.kr/store/store_map.do"
driver.get(url = URL)

driver.implicitly_wait(time_to_wait=10)

In [711]:
# 어떤 요소를 가지고 값을 선택할지에 대한 라이브러리
from selenium.webdriver.common.by import By
import time

location_search = driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a")
# "지역 검색" 으로 이동
location_search.click()
                            

In [712]:
location_list = driver.find_elements(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article > div.loca_step1 > div.loca_step1_cont > ul > li > a")

for location in location_list:
    print(location.text, end = ' ')

In [713]:
# 페이지의 "지역 검색" 클릭
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a").click()

In [714]:
# 페이지의 "서울 전체" 매장으로 이동
driver.find_element(By.CSS_SELECTOR, "#mCSB_2_container > ul > li:nth-child(1) > a").click()

### 서울 전체 매장 데이터 크롤링

In [715]:
data_location_list = []
data_name_list = []
data_lat_list = []
data_long_list = []
매장_list = driver.find_elements(By.CSS_SELECTOR, "#mCSB_3_container > ul > li")

for 매장 in 매장_list:
    data_location_list.append(location.text)
    # 매장의 이름, 위도, 경도 값을 list에 저장
    data_name_list.append(매장.get_attribute("data-name"))
    data_lat_list.append(매장.get_attribute("data-lat"))
    data_long_list.append(매장.get_attribute("data-long"))


print(data_name_list[0:10])
print(data_lat_list[0:10])
print(data_long_list[0:10])

In [716]:
import pandas as pd

df = pd.DataFrame()

df["이름"] = data_name_list
df["lat"] = data_lat_list
df["long"] = data_long_list

df.to_csv("starbucks_seoul_data.csv")
# df.to_excel("starbucks_seoul_data.xlsx")

### 개별 체크박스 크롤링

In [717]:
# 페이지의 "옵션 선택" 으로 이동
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > header > p > a").click()

체크박스 목록 가져오기

In [718]:
# ✅ "위치 및 시설" 섹션 (`dl.opt_select_dl3`) 찾기
checkbox_section = driver.find_element(By.CSS_SELECTOR, "#mCSB_7_container > dl.opt_select_dl3")

# ✅ 섹션 내 모든 label 요소 가져오기
labels = checkbox_section.find_elements(By.CSS_SELECTOR, "dd > label")

# ✅ 스크롤 가능한 div 찾기 (스크롤을 적용할 대상)
#mCSB_7_dragger_vertical
scrollable_div = driver.find_element(By.CSS_SELECTOR, "#mCSB_7_dragger_vertical")

# ✅ 체크박스가 화면에 보이도록 하나씩 `scrollIntoView()`
for label in labels:
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", label)
    time.sleep(0.5)  # 스크롤 후 로딩 대기

# ✅ 체크박스 이름을 저장할 리스트
checkbox_name_list = [label.text.strip() for label in labels]

# ✅ 결과 출력
print("✅ 저장된 체크박스 이름 목록:")
print(checkbox_name_list)

In [719]:
# 검색할 옵션 리스트
options_to_select = ["대학가", "터미널/기차역", "병원", "지하철 인접"]

# 결과 저장 리스트
all_data = []


옵션 check

In [720]:
import pandas as pd
import os

save_dir = "data"  # 저장할 디렉토리

# 디렉토리 없으면 생성
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print(f"📂 저장 경로: {os.path.abspath(save_dir)}")



In [721]:
count = 0  # 클릭 횟수를 추적하는 변수

for option in options_to_select:
    try:
        print(f"\n🔎 현재 옵션: {option}")

        # ✅ 1. 옵션 선택 창 다시 열기 (필수)
        try:
            options_toggle = driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > header > p > a")

            # 첫 번째 옵션 선택 시에는 조건 확인, 이후에는 무조건 클릭
            if count == 0:
                print("옵션 창이 이미 열려 있음, 클릭 생략")
            else:
                options_toggle.click()
                time.sleep(2)

            count += 1  # 클릭 횟수 증가

        except Exception as e:
            print(f"❌ 옵션 선택 창 열기 오류 발생: {e}")


        # ✅ 2. "위치 및 시설" 섹션 (`dl.opt_select_dl3`) 찾기
        checkbox_section = driver.find_element(By.CSS_SELECTOR, "#mCSB_7_container > dl.opt_select_dl3")

        # ✅ 3. 섹션 내 모든 label 요소 가져오기
        checkboxes = checkbox_section.find_elements(By.CSS_SELECTOR, "input[type='checkbox']")
        labels = checkbox_section.find_elements(By.CSS_SELECTOR, "dd > label")  # 라벨도 가져옴

        # ✅ 4. 스크롤 가능한 div 찾기 (스크롤을 적용할 대상)
        #mCSB_7_dragger_vertical
        scrollable_div = driver.find_element(By.CSS_SELECTOR, "#mCSB_7_dragger_vertical")

        # ✅ 5. 체크박스가 화면에 보이도록 하나씩 `scrollIntoView()`
        for label in labels:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", label)
            time.sleep(0.5)  # 스크롤 후 로딩 대기

        # ✅ 6. 체크박스와 이름을 매칭하는 딕셔너리 생성
        checkbox_dict = {label.text.strip(): checkbox for label, checkbox in zip(labels, checkboxes) if label.text.strip()}

        # ✅ 7. 체크박스 이름을 저장할 리스트
        checkbox_name_list = list(checkbox_dict.keys())

        # 결과 출력
        print("✅ 저장된 체크박스 이름 목록:")
        print(checkbox_name_list)

        # ✅ 8. 기존 체크박스 정보 재사용하여 모든 체크 해제
        for name, checkbox in checkbox_dict.items():
            if checkbox.is_selected():
                driver.execute_script("arguments[0].click();", checkbox)
                time.sleep(1)

        # ✅ 9. 특정 옵션 선택 (스크롤 후 클릭)
        if option in checkbox_dict:
            # block 뒤에 start, center, end 등으로 화면 위치 조정
            driver.execute_script("arguments[0].scrollIntoView({block: 'end'});", checkbox_dict[option])
            # driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", checkbox_dict[option])
            time.sleep(1)
            driver.execute_script("arguments[0].click();", checkbox_dict[option])
            time.sleep(1)


        # ✅ 10. "필터 적용" 버튼 클릭
        apply_button = driver.find_element(By.CSS_SELECTOR, "#storeMap > form > fieldset > div > ul > li.li2 > a")
        apply_button.click()

        time.sleep(5)

        # ✅ 11. 적용된 필터의 매장 목록 크롤링
        filtered_stores = driver.find_elements(By.CSS_SELECTOR, "#mCSB_3_container > ul > li")

        # ✅ 12. 매장 정보 저장
        filtered_store_data = []
        for store in filtered_stores:
            store_name = store.get_attribute("data-name")
            store_lat = store.get_attribute("data-lat")
            store_long = store.get_attribute("data-long")
            filtered_store_data.append({
                "옵션": option,
                "매장명": store_name,
                "위도": store_lat,
                "경도": store_long
            })

        all_data.extend(filtered_store_data)
        print(f"✅ [{option}] - {len(filtered_store_data)}개 매장 크롤링 완료")

        # DataFrame 생성
        df_option = pd.DataFrame(filtered_store_data)

        # ✅ 개별 옵션별 파일명 설정
        if "/" in option or " " in option:
            safe_option = option.replace("/", "_").replace(" ", "")
        else:
            safe_option = option  # 변경할 필요 없으면 원본 유지
        
        # 개별 옵션별 파일명 설정
        file_name_csv = f"{save_dir}/스타벅스_{safe_option}.csv"
        file_name_excel = f"{save_dir}/스타벅스_{safe_option}.xlsx"

        # CSV & Excel 저장
        df_option.to_csv(file_name_csv, index=False, encoding="utf-8-sig")  # CSV 저장
        df_option.to_excel(file_name_excel, index=False)  # Excel 저장

        print(f"✅ [{safe_option}] - {len(filtered_store_data)}개 매장 크롤링 완료 및 저장 완료 (CSV, Excel)")

    except Exception as e:
        print(f"❌ [{option}] - 오류 발생: {e}")

# 최종 출력
print(f"📂 모든 옵션별 데이터가 '{save_dir}' 폴더에 저장되었습니다.")